In [7]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.decomposition import PCA
from time import time

In [9]:
def all_same(items):
  return len(set(items)) == 1

In [10]:
data = pd.read_csv("./train.csv")
data_t = pd.read_csv("./test.csv")

In [11]:
feature_cols = list(data.columns[1:])
feature_cols_t = list(data_t.columns[1:])

In [12]:
target_col = data.columns[0]
target_col_t = data_t.columns[0]

In [13]:
X = data[feature_cols]
y = data[target_col]
X_t = data_t[feature_cols_t]
y_t = data_t[target_col_t]

In [14]:
pca = PCA(n_components=60).fit(X)
pca_t = PCA(n_components=60).fit(X_t)
# Transform the data using the PCA fit above
X = pca.transform(X)
y = y.values
X_t = pca_t.transform(X_t)
y_t = y_t.values

In [15]:
X_train = X
y_train = y
X_test = X_t
y_test = y_t
print(X_test.shape,y_test.shape)
# **************************************
# **************************************
models = {}

(744, 60) (744,)


In [16]:
knn_clf = KNeighborsClassifier(n_neighbors=3, p=2)
knn_clf.fit(X_train,y_train)
# Initialize the array of predicted labels
# y_pred = []
y_pred = np.empty(len(y_test),dtype=int)

In [17]:
start = time()

In [18]:
kneighbors = knn_clf.kneighbors(X_test,n_neighbors=5, return_distance=False)
# For each set of neighbors indices
for idx, indices in enumerate(kneighbors):
  # Find the actual training samples & their labels
  neighbors = [X_train[i] for i in indices]
  neighbors_labels = [y_train[i] for i in indices]
  
  # if all labels are the same, use it as the prediction
  if all_same(neighbors_labels):
    y_pred[idx] = neighbors_labels[0]
  else:
    # else fit a SVM classifier using the neighbors, and label the test samples
    svm_clf = svm.SVC(C=0.5, kernel='rbf', decision_function_shape='ovo', random_state=42)
    svm_clf.fit(neighbors, neighbors_labels)
    label = svm_clf.predict(X_test[idx].reshape(1, -1))
    
    y_pred[idx] = label
end = time()
# for i in range(len(y_test)):
#   print(str(i) + ". expected : " + str(y_test[i]) + " output : " + str(y_pred[i]))

In [19]:
print(accuracy_score(y_test, y_pred))
print("Made predictions in {:.4f} seconds.".format(end - start))

0.06989247311827956
Made predictions in 0.7860 seconds.
